
# Capítulo 2 — Fundamentos de Python e Ambiente (Scikit‑learn)

**Curso:** CECIERJ – IA e ML para Soluções Práticas  
**Objetivo:** configurar ambiente, revisar Python essencial, e praticar NumPy, pandas, matplotlib e um *hello world* do Scikit‑learn com *pipeline* e validação.

## Como preparar o ambiente (fora do notebook)
**Windows PowerShell**
```pwsh
python -m venv .venv
.\.venv\Scripts\activate
pip install -r env/requirements.txt
python -m ipykernel install --user --name cecierj-ia-ml
```
**Linux/macOS**
```bash
python3 -m venv .venv
source .venv/bin/activate
pip install -r env/requirements.txt
python3 -m ipykernel install --user --name cecierj-ia-ml
```
Abra o Jupyter Lab e selecione o *kernel* **cecierj-ia-ml**.


In [ ]:

# Checagem de versões
import sys
print("Python:", sys.version)
try:
    import numpy, pandas, sklearn, matplotlib
    print("numpy:", numpy.__version__)
    print("pandas:", pandas.__version__)
    print("scikit-learn:", sklearn.__version__)
    print("matplotlib:", matplotlib.__version__)
except Exception as e:
    print("Dependências ausentes:", e)



## 1) Python Essencial
Tipos, estruturas, funções, erros e dicas de escrita.


In [ ]:

# Tipos e estruturas
x = 3
nums = [1, 2, 3]
d = {"curso": "CECIERJ", "modulo": 2}
s = {1, 2, 2, 3}
t = (10, 20)

print(type(x), type(nums), type(d), type(s), type(t))
print(nums[0], d["curso"], 2 in s)


In [ ]:

# Compreensões, desempacotamento e f-strings
quad = [n*n for n in range(6)]
a, b, *resto = [10, 20, 30, 40]
print(quad, a, b, resto)
nome, cap = "Ricardo", 2
print(f"Capítulo {cap} — Olá, {nome}!")


In [ ]:

# Funções com anotações e docstring
from typing import List

def media(v: List[float]) -> float:
    """Retorna a média de uma lista numérica."""
    if not v:
        raise ValueError("Lista vazia")
    return sum(v) / len(v)

print(media([1,2,3]))


In [ ]:

# Tratamento de erros
try:
    print(media([]))
except ValueError as e:
    print("Erro esperado:", e)



## 2) NumPy em 5 minutos
Vetores/ Matrizes, *broadcasting*, multiplicação matricial e reprodutibilidade.


In [ ]:

import numpy as np

rng = np.random.default_rng(42)  # semente
a = rng.integers(0, 10, size=(3,3))
b = rng.integers(0, 10, size=(3,3))
print("a:\n", a, "\n b:\n", b)

# Soma com broadcasting
print("a + 1:\n", a + 1)

# Produto matricial
print("a @ b:\n", a @ b)

# Estatísticas
print("média de a:", a.mean(), "desvio pad:", a.std())



## 3) pandas em 5 minutos
DataFrames, tipos, *describe*, tratamento de ausentes e *encoding* simples.


In [ ]:

import pandas as pd
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)
df = data.frame.copy()
print(df.dtypes.head())
display(df.head())
display(df.describe())


In [ ]:

# Introduzindo ausentes artificialmente e tratando
df2 = df.copy()
df2.loc[df2.sample(frac=0.01, random_state=42).index, df2.columns[0]] = None
print("Ausentes antes:", df2.isna().sum().sum())
df2.fillna(df2.median(numeric_only=True), inplace=True)
print("Ausentes depois:", df2.isna().sum().sum())


In [ ]:

# Exemplo de one-hot encoding simples (categorias simuladas)
toy = pd.DataFrame({
    "bairro": ["Centro","Norte","Centro","Sul","Sul"],
    "quartos": [2,3,2,4,3],
    "preco": [300, 450, 320, 700, 500]
})
display(toy)
toy_enc = pd.get_dummies(toy, columns=["bairro"], drop_first=True)
display(toy_enc)



## 4) matplotlib rápido
Plot simples (sem estilos específicos).


In [ ]:

import matplotlib.pyplot as plt

plt.figure()
df['alcohol'].hist(bins=20)
plt.title("Distribuição da variável 'alcohol' (Wine)")
plt.xlabel("alcohol")
plt.ylabel("contagem")
plt.show()



## 5) Scikit‑learn *hello world*
- *train/test split*
- `Pipeline` (escalonamento + modelo)
- `cross_val_score` e *random_state* para reprodutibilidade


In [ ]:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import numpy as np

iris = load_iris(as_frame=True)
X = iris.frame.drop(columns=["target"])
y = iris.frame["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, multi_class="ovr", random_state=42))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipe, X, y, cv=cv, scoring="accuracy")
print("CV Accuracy média:", scores.mean())
pipe.fit(X_train, y_train)
print("Acurácia holdout:", pipe.score(X_test, y_test))



## 6) *ColumnTransformer* (numéricas + categóricas) — exemplo mínimo
Transforma diferentes colunas com *pipelines* distintos (numéricas e categóricas).


In [ ]:

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

dfc = pd.DataFrame({
    "bairro": ["Centro","Norte","Centro","Sul","Sul","Norte","Sul","Centro"],
    "quartos": [2,3,2,4,3,2,4,1],
    "area": [60, 80, 55, 120, 90, 50, 110, 40],
    "preco": [320, 480, 300, 800, 520, 260, 750, 220]
})
Xc = dfc[["bairro", "quartos", "area"]]
yc = dfc["preco"]

num_cols = ["quartos","area"]
cat_cols = ["bairro"]

ct = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(drop="first"), cat_cols),
])

reg = Pipeline([("prep", ct), ("lr", LinearRegression())])
Xc_tr, Xc_te, yc_tr, yc_te = train_test_split(Xc, yc, test_size=0.25, random_state=42)

reg.fit(Xc_tr, yc_tr)
pred = reg.predict(Xc_te)
mse = mean_squared_error(yc_te, pred)
print("MSE:", round(mse,2))



## 7) Salvando e carregando modelos (`joblib`)
Demonstração prática para *persistência*.


In [ ]:

import joblib, os
save_path = "/mnt/data/modelo_cap2.joblib"
joblib.dump(pipe, save_path)
print("Modelo salvo em:", save_path)

loaded = joblib.load(save_path)
print("Score após carregar:", loaded.score(X_test, y_test))



## Checklist de Reprodutibilidade
- Fixe *seeds* (`random_state`).
- Registre versões (Python/Libs).
- Use `Pipeline`/`ColumnTransformer` para evitar *leakage*.
- Versione código e dados.
- Documente métricas e decisões.
